# Lineární klasifikace obrázků CIFAR10

- Úkolem cvičení je naprogramovat lineární klasifikátor, který bude rozpoznávat objekty z datasetu CIFAR-10.
- Vstupem je RGB obrázek o rozměrech 32×32 pixelů a úkolem říci, který z 10 možných objektů (tříd) je na něm zachycen.
- Možnosti jsou tyto: letadlo, automobil, pták, kočka, jelen, pes, žába, kůň, loď, náklaďák.
- Použijeme přitom **lineární model**, **softmax křížovou entropii** a **gradient descent**.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')  # import tests, ans

import numpy as np
import matplotlib.pyplot as plt
import PIL
import torch
import torchvision

import ans
from tests import test_linear_classification

# Data

## Preprocessing

- Data převedeme z `PIL.Image` do `torch.Tensor`.
- Pro lepší numerické vlastnosti zároveň konvertujeme do rozsahu 0-1 a datového typu `torch.float32`, který je pro PyTorch výchozí. Stačí vydělit 255.
- Zároveň obrázky přetvarujeme do vektoru, takže výstup bude mít rozměr (batch_size, počet_pixelů).
- Targety pouze převedeme z `numpy.ndarray` do `torch.Tensor`.
- Celé předzpracování bude implementovat funkce `preprocess`, kterou budeme později volat pro každou dávku po jejím načtení v `BatchLoader`u.

### TODO: implementuje funkci `preprocess`.

In [3]:
def preprocess(img: PIL.Image) -> torch.Tensor:
    """
    Args:
        img: image of type PIL.Image
    Returns:
        x: 1-dimensional tensor; shape (num_pixels * num_channels,), dtype float32
    """
    
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return x

In [ ]:
test_linear_classification.TestPreprocess.eval(preprocess_fn=preprocess)

## Trénovací a validační datasety

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=preprocess)
val_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=preprocess)

In [ ]:
train_dataset[5]

## Načítání po dávkách

- Načítání dat po dávkách bude zajišťovat třída `ans.data.BatchLoader`.
- Třída implementuje metodu `__iter__` a je tedy možné ji použít jako zdroj např. pro `for` cyklus následujícím způsobem:
  ``` python
  train_loader = ans.data.BatchLoader(
      dataset,  # napr. torchvision.datasets.CIFAR10
      batch_size = 100,
      shuffle = True,  # nahodne poradi
  )
  for inputs, targets in train_loader:
      # inputs ... shape (100, 32, 32, 3)
      # targets ... shape (100,)
      ...
  ```

### TODO: implementujte metodu [`ans.data.BatchLoader.__iter__`](../ans/data.py).

In [ ]:
train_loader = ans.data.BatchLoader(
    train_dataset,
    batch_size = 5,
    shuffle = True,
)
train_loader

In [ ]:
inputs, targets = next(iter(train_loader))
print(type(inputs), inputs.shape, inputs.dtype, inputs.min(), inputs.max())
print(type(targets), targets.shape, targets.dtype, targets.min(), targets.max())

In [ ]:
test_linear_classification.TestBatchLoader.eval()

# Lineární softmax klasifikátor

- Lineární softmax klasifikátor optimalizuje křížovou entropii
  $$
  l_n = -\log\frac{\exp{s_{n,y_n}}}{\sum_{k=1}^{K}{\exp{s_{n,k}}}}
  $$
  kde skóre $\boldsymbol{s}_n$ jsou vypočteny jako lineární (afinní) funkce
  $$
  \boldsymbol{s}_n = \boldsymbol{x}_n \cdot \boldsymbol{w} + \boldsymbol{b}
  $$
- Klasifikátor bude implementován jako třída [`ans.classification.LinearSoftmaxModel`](../ans/classification.py).
- Třída obsahuje parametry klasifikátoru jako atributy  
  | atribut  | značení                               | rozměr       |
  |----------|---------------------------------------|--------------|
  | `weight` | $\boldsymbol{w} = [w_{d,k}]$          | $D \times K$ |
  | `bias`   | $\boldsymbol{b} = [b_1, \ldots, b_K]$ | $K$          |
- Metoda `__init__` inicializuje váhovou matici a vektor biasů na náhodná čísla  
  ``` python
  def __init__(self, in_size, out_size, weight_scale=1e-3) -> None:
    # self.weight = ...
    # self.bias = ...
  ```
- Metoda `train_step` provede jeden krok učení (včetně updatu parametrů) na zadané (mini)dávce a vrátí vypočtený loss a predikované skóre  
  ``` python
  def train_step(self, inputs, targets, learning_rate=1e-3):
    # predikce skore na davce
    # vypocet lossu (softmax krizova entropie)
    # vypocet gradientu
    # update parametru metodou nejvetsiho spadu
    # return loss, skore
  ```
- Metoda `val_step` provede pouze predikci (tedy bez updatu parametrů) na zadané (mini)dávce a vrátí vypočtený loss a predikované skóre  
  ``` python
  def val_step(self, inputs, targets, learning_rate=1e-3):
    # predikce skore na davce
    # vypocet lossu (softmax krizova entropie)
    # return loss, skore
  ```

**Poznámky k `train_step`**

- Data v tensorech jsou uložena *po řádcích*. Výpočet skóre je proto oproti přednášce *transponovaný*, tj.
  $$\boldsymbol{s}_n = \boldsymbol{x}_n \cdot \boldsymbol{w} + \boldsymbol{b}$$
  kde
  - $\boldsymbol{s}_n = [s_{n,1},\ldots,s_{n,K}]$ je *řádkový* vektor skóre pro $n$-tý vzorek (obrázek) $\boldsymbol{x}_n$ a každou z $K$ tříd
  - $\boldsymbol{x}_n = [x_{n,1},\ldots,x_{n,D}]$ je $n$-tý vzorek (obrázek) v dávce reprezentovaný jako (řádkový) vektor s rozměrem $D$.
- Celkový loss pro dávku vypočtěte jako *průměr* dílčích lossů za jednotlivé obrázky
  $$
  l = \frac{1}{N} \cdot \sum_{n=1}^{N}{l_n}
  $$
- Parametry updatujte základní metodou největšího spádu. Nepoužívejte momentum ani adaptivní metody.

### TODO: implementuje metodu [`ans.classification.LinearSoftmaxModel.__init__`](../ans/classification.py).

In [ ]:
test_linear_classification.TestInit.eval()

### TODO: implementuje funkci [`ans.classification.LinearSoftmaxModel.train_step`](../ans/classification.py).

In [ ]:
test_linear_classification.TestTrainStepSoftmax.eval()

### TODO: implementuje funkci [`ans.classification.LinearSoftmaxModel.val_step`](../ans/classification.py).

In [ ]:
test_linear_classification.TestValStepSoftmax.eval()

# Úspěšnost: accuracy

- Kromě lossu budeme pro lepší orientaci měřit i přesnost (accuracy), byť tuto veličinu nebudeme přímo optimalizovat.
- Accuracy (správnost) $a$ vypočteme jako
  $$
  a = \frac{1}{N} \cdot \sum_{n=1}^{N}{\mathbb{1}(\hat{y}_n = y_n)}
  $$

### TODO: implementujte funkci [`ans.classification.accuracy`](../ans/classification.py).

In [ ]:
test_linear_classification.TestAccuracy.eval()

# Trénování klasifikátoru

## Funkce `train_epoch`

- Vytvořímě funkci `train_epoch`, která převezme model a dataloader a zavolá `train_step` modelu na včechny dávky v dataloaderu.  
  Funkce musí umět převzít `learning_rate` a předat jí do volané `train_step`.

### TODO: implementujte funkci [`ans.classification.train_epoch`](../ans/classification.py).

In [ ]:
test_linear_classification.TestTrainEpoch.eval()

## Funkce `validate`

- Analogicky k ní vytvoříme ještě druhou funkci `validate`, která provede pouze predikci modelu na dataloaderu a vrátí průměrný loss a accuracy.

### TODO: implementujte funkci [`ans.classification.validate`](../ans/classification.py).

In [ ]:
test_linear_classification.TestValidate.eval()

## Hlavní cyklus trénovaání

- Finálním úkolem je natrénovat lineární klasifikátor na co největší *validační* accuracy.
- **Nejlepší natrénovaný model uložte do souboru  `output/linear_softmax_weights.pt`**.
- Ukládejte pouze parametry `weights` a `bias`, tj. nikoliv celý objekt `LinearSoftmaxModel` a to následovně:
  ``` python
  torch.save(
    dict(weight=model.weight, bias=model.bias),
    '../output/linear_classification_softmax_weights.pt'
  )
  ```
- Model se později načte a v rámci testů vyhodnotí na validační sadě CIFAR-10. Je proto nutné dodržet cesty i formát ukládání.

### TODO: Natrénujte lineární softmax klasifikátor tak, aby dosáhl alespoň 40 % *validační* accuracy.

In [ ]:
ans.utils.seed_everything(0)

# hyperparametry
batch_size = ...
learning_rate = ...
num_epochs = ...

# dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=preprocess)
val_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=preprocess)

# loadery
train_loader = ans.data.BatchLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = ans.data.BatchLoader(val_dataset, batch_size=batch_size, shuffle=False)

# model
model = ans.classification.LinearSoftmaxModel(...)

# validace pred trenovanim (sanity check, loss by mel byt cca 2.30)
train_loss, train_acc = ...
val_loss, val_acc = ...
print(f"after init: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

# optimalizace
for epoch in range(num_epochs):
    # trenovani
    ...
        
    # validace
    train_loss, train_acc = ...
    val_loss, val_acc = ...
    print(f"epoch {epoch + 1}: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

    # uloz nejlepsi model
    ...

In [ ]:
test_linear_classification.TestSoftmaxValAccuracy.eval(preprocess_fn=preprocess)

# Support Vector Machine (SVM)

- SVM je softmaxu velmi podobné. Z pohledu neuronových sítí se liší pouze způsobem výpočtu lossu.
- Místo (softmax) cross entropy optimalizuje hinge loss
  $$l_n = \sum_{c\ne y_n}\max(0, 1 + s_{n,c} - s_{n,y_n})$$
  kde:
  - $y_n \in \{1, \ldots, C\}$ je správný index třídy (celé číslo) na vzorku (obrázku) $\boldsymbol{x}_n$
  - $s_{n,i}$ je skóre (logity) predikované lineárním klasifikátorem pro $n$-tý obrázek a $i$-tou třídu.
- Gradient na $c$-tý sloupec $\boldsymbol{w}_{:,c}$ váhové matice $\boldsymbol{w}$ pak je
  $$
  \frac{\partial l_n}{\partial \boldsymbol{w}_{:,c}} = \begin{cases}
      -\sum_{c\ne y_n}\mathbb{1}(1 + s_{n,c} - s_{n,y_n} > 0)\cdot\boldsymbol{x}_n & \textrm{pokud} & c = y_n \\
      \mathbb{1}(1 + s_{n,c} - s_{n,y_n} > 0) \cdot \boldsymbol{x}_n & \textrm{pokud} & c \ne y_n
  \end{cases}
  $$
  kde
  - $\mathbb{1}(\cdot) = 1$, pokud podmínka $\cdot$ je splněna, jinak $\mathbb{1}(\cdot) = 0$
- Gradient na bias je
  $$
  \frac{\partial l_n}{\partial b_c} = \begin{cases}
      -\sum_{c\ne y_n}\mathbb{1}(1 + s_{n,c} - s_{n,y_n} > 0) & \textrm{pokud} & c = y_n \\
      \mathbb{1}(1 + s_{n,c} - s_{n,y_n} > 0) & \textrm{pokud} & c \ne y_n
  \end{cases}
  $$

### TODO: implementuje funkci [`ans.classification.LinearSVMModel.train_step`](../ans/classification.py).

In [ ]:
test_linear_classification.TestTrainStepSVM.eval()

### TODO: implementuje funkci [`ans.classification.LinearSVMModel.val_step`](../ans/classification.py).

In [ ]:
test_linear_classification.TestValStepSVM.eval()

## Hlavní cyklus trénování

- Nejlepší model uložte jako
  ``` python
  torch.save(
    dict(weight=model.weight, bias=model.bias),
    '../output/linear_classification_svm_weights.pt'
  )
  ```

### TODO: Natrénujte lineární SVM klasifikátor tak, aby dosáhl alespoň 39 % *validační* accuracy.

In [ ]:
ans.utils.seed_everything(0)

# hyperparametry
batch_size = ...
learning_rate = ...
num_epochs = ...

# dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, transform=preprocess)
val_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, transform=preprocess)

# loadery
train_loader = ans.data.BatchLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = ans.data.BatchLoader(val_dataset, batch_size=batch_size, shuffle=False)

# model
model = ans.classification.LinearSVMModel(...)

# validace pred trenovanim (sanity check, loss by mel byt cca 9)
train_loss, train_acc = ...
val_loss, val_acc = ...
print(f"after init: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

# optimalizace
for epoch in range(num_epochs):
    # trenovani
    ...
    
    # validace
    train_loss, train_acc = ...
    val_loss, val_acc = ...
    print(f"epoch {epoch + 1}: train_loss={train_loss:.5f}, train_acc={train_acc:.3f}, val_loss={val_loss:.5f}, val_acc={val_acc:.3f}")

    # uloz nejlepsi model
    ...

In [ ]:
test_linear_classification.TestSVMValAccuracy.eval(preprocess_fn=preprocess)